<a href="https://colab.research.google.com/github/Eugeneantientropy/ML100Days/blob/main/HW_Day_88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

參考範例程式碼Day088_CB_CustomizedCallbacks.ipynb，請嘗試寫一個 callback 用來記錄各類別在訓練過程中，對驗證集的 True Positive 與 True Negative



作業請提交Day088_HW.ipynb

In [2]:
import os
import keras
import numpy as np
from keras.layers import BatchNormalization
from keras.callbacks import Callback
from sklearn.metrics import f1_score, confusion_matrix

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

train, test = keras.datasets.cifar10.load_data()

## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)


"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)

    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units,
                                   activation="relu",
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units,
                                   activation="relu",
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)

    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)

    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

class TP_TN_Tracker(Callback):
    def __init__(self, x_val, y_val, threshold=0.5):
        super(TP_TN_Tracker, self).__init__()
        self.x_val = x_val
        self.y_val = y_val
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        y_true = self.y_val.argmax(axis=1)
        y_pred_proba = self.model.predict(self.x_val, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)

        # 計算 weighted F1-score
        f1sc_value = f1_score(y_true, y_pred, average="weighted")
        logs["val_f1sc"] = f1sc_value

        # 計算 confusion matrix 並取得 TP 和 TN
        cm = confusion_matrix(y_true, y_pred)
        for cls_idx in range(cm.shape[0]):
            TP = cm[cls_idx, cls_idx]
            FN = np.sum(cm[cls_idx, :]) - TP
            FP = np.sum(cm[:, cls_idx]) - TP
            TN = np.sum(cm) - (TP + FN + FP)

            logs[f"val_tp_class_{cls_idx}"] = TP
            logs[f"val_tn_class_{cls_idx}"] = TN
            print(f"Class {cls_idx}: TP={TP}, TN={TN}")

        print(f"\nEpoch {epoch + 1} - Weighted F1 Score: {f1sc_value:.4f}")

# 使用範例
log_tp_tn = TP_TN_Tracker(x_val=x_test, y_val=y_test, threshold=0.5)

model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(learning_rate=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=(x_test, y_test),
          shuffle=True,
          callbacks=[log_tp_tn]
         )

# 取得 f1sc 紀錄
valid_f1sc = model.history.history['val_f1sc']
print(f"Validation F1 Score per epoch: {valid_f1sc}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer1 (Dense)                │ (None, 512)                 │       1,573,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer2 (Dense)                │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden_layer3 (Dense)                │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,742,474 (6.65 MB)

 Trainable params: 1,740,682 (6.64 MB)

...

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 3072))
  warnings.warn(msg)


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.1850 - loss: 2.6204

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 3072))
  warnings.warn(msg)


Class 0: TP=876, TN=3684
Class 1: TP=73, TN=8822
Class 2: TP=17, TN=8945
Class 3: TP=79, TN=8725
Class 4: TP=139, TN=8672
Class 5: TP=171, TN=8521
Class 6: TP=52, TN=8875
Class 7: TP=20, TN=8922
Class 8: TP=198, TN=7633
Class 9: TP=40, TN=8866

Epoch 1 - Weighted F1 Score: 0.1253
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 165ms/step - accuracy: 0.1865 - loss: 2.6135 - val_accuracy: 0.1665 - val_loss: 2.1794 - val_f1sc: 0.1253 - val_tp_class_0: 876.0000 - val_tn_class_0: 3684.0000 - val_tp_class_1: 73.0000 - val_tn_class_1: 8822.0000 - val_tp_class_2: 17.0000 - val_tn_class_2: 8945.0000 - val_tp_class_3: 79.0000 - val_tn_class_3: 8725.0000 - val_tp_class_4: 139.0000 - val_tn_class_4: 8672.0000 - val_tp_class_5: 171.0000 - val_tn_class_5: 8521.0000 - val_tp_class_6: 52.0000 - val_tn_class_6: 8875.0000 - val_tp_class_7: 20.0000 - val_tn_class_7: 8922.0000 - val_tp_class_8: 198.0000 - val_tn_class_8: 7633.0000 - val_tp_class_9: 40.0000 - val_tn_class_9: 8866.0000
Epoch 2/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 0

NameError: name 'train_loss' is not defined